In [66]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import string
import matplotlib.cm as cm
import os
from scipy import ndimage

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pi']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [67]:
rll=0.139 # Radio geométrico de las llantas en traseras (m)
refc=0.98*rll # Radio efectivo llantas traseras (m)
rin=0.0631 #radio interno del neumático (m)

# Datos eje llantas
reje=0.02 # Radio eje de las llantas (m)
meje=3.832 # Masa del eje (kg)


mllt=1.43 # Masa llantas traseras (kg)
mlld=1.20 # Masa llantas delanteras (kg)
rlld=0.129 # Radio llantas delanteras (m)
rind=0.0631 # Radio rin llantas delanteras (m)
mbt=0.7*mllt # El porcentaje de la masa total que pesa la parte "banda" de la llanta trasera
mdt=0.3*mllt # El porcentaje de la masa total que pesa la parte "disco" de la llanta trasera
mbd=0.7*mlld # El porcentaje de la masa total que pesa la parte "banda" de la llanta delantera
mdd=0.3*mlld # El porcentaje de la masa total que pesa la parte "disco" de la llanta delantera
illd=(mbd*rlld)+(mdd*(rlld**2-rind**2))/2 # Inercia 1 llanta delantera (kg-m^2)
illt=(mbt*rll)+(mdt*(rll**2-rin**2))/2 # Inercia 1 llanta trasera (kg-m^2)

ieje=(meje*reje**2)/2.0 # Inercia del eje de las llantas (kg-m^2)

theta=0##angulo de inclinacion de la pista
nt=0.95 #eficiencia de la transmision
ncelectrico=3.2
nccombustion=8
r=refc#radio de las llantas
m=147 #masa vehiculo
rho=0.95 #densidad del aire
af=0.575 #area frontal del vehiculo
cd=0.804 #coeficiente de arrastre
inw=illt#inercia de las llantas
ind=ieje#inercia del eje
ine=5.2E-3#inercia del motor
pi=3.1459265 #coeficiente de arrastre
deltat1=0.01;
deltat2=0.05;


###Punto 1
funcion para tomar los datos del vbox con entradas de la direccion del archivo y salida arreglos con los datos deseados

In [68]:
def datos(base,data):
 with open(os.path.join(base,data), 'r') as vboxdata:
    section = []
    time = []
    lat = []
    long = []
    vel = []
    height = []
    col_names=[]
    cont=0
    for line in vboxdata:
        if line[0] == '[':
            section = str.replace(line, "\n","")
            cont=0
            continue
        
        if section == '[data]':
            linesplit = str.replace(line, " \n","").split(' ')
            time.append(float(linesplit[1][0:2])*3600+float(linesplit[1][2:4])*60+float(linesplit[1][4:]))
            lat.append(float(linesplit[2]))
            long.append(float(linesplit[3]))
            vel.append(float(linesplit[4]))
            height.append(float(linesplit[6]))
       

    time = np.asarray(time)
    lat = np.asarray(lat)
    long = np.asarray(long)
    vel = np.asarray(vel) 
    height = np.asarray(height)
    time_base = time[0]
    time = np.asarray(time) - time_base
    return (time,lat,long,vel)

In [69]:
def acelfil(vel,ddt,num):
    return ndimage.gaussian_filter1d(vel, sigma=num, order=1, mode='wrap') / ddt

In [70]:
def velfil(vel,num):
    return ndimage.gaussian_filter1d(vel, sigma=num, order=0, mode='wrap')

In [71]:
def da(vel):#funcion para calcular la fuerza de arrastre a una velocidad dada
 resp=0.5*rho*af*cd*(vel**2)
 return (resp)

In [72]:
def fr(vel,mtot):#funcion para calcular la fuerza de rodadura a una velocidad dada
    vkmh=vel*3.6
    w=mtot*9.8
    fr=0.01*(1+(vkmh/100.0)) 
    return w*fr*cos(theta)

In [73]:
def motor(vel,accel,rel):
 mr=(inw+ind+(ine*(rel**2)))/(r**2)##calculo de la masa rotacional
 mtot=m+mr##calculo de la masa total
 fuerzas=(da(vel)+fr(vel,mtot)+(mtot*accel))
 torque=fuerzas*r/(rel*nt)
 velang=vel*rel*9.5492/r
 return (velang,torque)


In [74]:
def recorrido(vel,tvel):
    sol=[]
    largo=len(vel)
    tamanio=(np.linspace(1,largo-1,largo)).astype(int)
    for i in tamanio:
     distancia=np.trapz(vel[0:i],tvel[0:i])
     sol.append(distancia)
    return(sol)

In [75]:
def motortotal(vel,acel,rel):
    torque=[]
    omega=[]
    largo=len(vel)
    tamanio=(np.linspace(0,largo-1,largo)).astype(int)
    for i in tamanio: 
     actual=motor(vel[i],acel[i],rel)
     omega.append(actual[0])
     torque.append(actual[1])
    return(omega,torque)
      

In [76]:
gasolina1=datos('Data','VBOX0019.VBO')
electrico1=datos('data_electrico','VBS__047.VBO')

timegas1=gasolina1[0]
velogas1=velfil(gasolina1[3]/3.6,30)
latgas1=gasolina1[1]
longgas1=gasolina1[2]


timeele1=electrico1[0]
veloele1=velfil(electrico1[3]/3.6,10)
latele1=electrico1[1]
longele1=electrico1[2]



In [77]:
inicio1 = 3190
puntos1 = 4800
inicio2=340
puntos2=1012

dist1=recorrido(velogas1[inicio1:inicio1+puntos1],timegas1[inicio1:inicio1+puntos1])
dist2=recorrido(veloele1[inicio2:inicio2+puntos2],timeele1[inicio2:inicio2+puntos2])
vel1=velogas1[inicio1:inicio1+puntos1]
vel2=veloele1[inicio2:inicio2+puntos2]
acel1=acelfil(velogas1[inicio1:inicio1+puntos1], deltat1,30)
acel2=acelfil(veloele1[inicio2:inicio2+puntos2], deltat2,10)
tiemp1=timegas1[inicio1:inicio1+puntos1]-timegas1[inicio1]
tiemp2=timeele1[inicio2:inicio2+puntos2]-timeele1[inicio2]
torq1=motortotal(vel1,acel1,nccombustion)[1]
torq2=motortotal(vel2,acel2,ncelectrico)[1]
velang1=motortotal(vel1,acel1,nccombustion)[0]
velang2=motortotal(vel2,acel2,ncelectrico)[0]
pot1=(np.asarray(velang1)*np.asarray(torq1))/9.58
pot2=np.asarray(velang2)*np.asarray(torq2)/9.58



plt.plot(latgas1[inicio1:inicio1+puntos1],longgas1[inicio1:inicio1+puntos1])
plt.plot(latele1[inicio2:inicio2+puntos2],longele1[inicio2:inicio2+puntos2])
plt.title('Grafica 1. mapa de la pista en los dos recorridos')
pylab.ylabel("latitud")
pylab.xlabel("Longitud")
plt.axis('off')
plt.grid()
pylab.savefig(os.path.join('graf1.png'), dpi=600)
plt.close()

plt.plot(dist1,tiemp1,label='Gasolina')
plt.plot(dist2,tiemp2,label='Electrico')
plt.title('Grafica 2. tiempo vs distancia recorrida')
pylab.ylabel("Tiempo [s]")
pylab.xlabel("Posicion en la pista [m]")
plt.grid()
pylab.savefig(os.path.join('graf2.png'), dpi=600)
plt.close()


plt.plot(dist1,vel1,label='Gasolina')
plt.plot(dist2,vel2,label='Electrico')
plt.title('Grafica 3. Velocidad vs distancia recorrida')
pylab.ylabel("velocidad [m/s]")
pylab.xlabel("Distancia recorrida [m]")
plt.grid()
pylab.legend(loc='upper left') 
pylab.savefig(os.path.join('graf3.png'), dpi=600)
plt.close()

plt.plot(dist1[60:-60],acel1[60:-60],label='Gasolina')
plt.plot(dist2[20:-20],acel2[20:-20],label='Electrico')
plt.title('Grafica 4. Aceleracion vs distancia recorrida')
pylab.ylabel("Aceleracion [m/s]")
pylab.xlabel("Distancia recorrida [m]")
plt.grid()
pylab.legend(loc='lower center') 
pylab.savefig(os.path.join('graf4.png'), dpi=600)
plt.close()

plt.plot(dist1[60:-60],torq1[60:-60],label='Gasolina')
plt.plot(dist2[20:-20],torq2[20:-20],label='Electrico')
plt.title('Grafica 5. Torque del motor vs Distancia recorrida')
pylab.ylabel("Torque [N*m]")
pylab.xlabel("Distancia recorrida [m]")
plt.grid()
pylab.legend(loc='lower left') 
pylab.savefig(os.path.join('graf5.png'), dpi=600)
plt.close()

plt.plot(dist1,velang1,label='Gasolina')
plt.title('Grafica 6. Velocidad del motor vs distancia recorrida-gasolina')
pylab.ylabel("Velocidad angular [RPM]")
pylab.xlabel("Posicion en la pista [m]")
plt.grid() 
pylab.savefig(os.path.join('graf6.png'), dpi=600)
plt.close()

plt.plot(dist2,velang2,label='Electrico')
plt.title('Grafica 7. Velocidad del motor vs distancia recorrida-electrico')
pylab.ylabel("Velocidad angular [RPM]")
pylab.xlabel("Posicion en la pista [m]")
plt.grid()
pylab.savefig(os.path.join('graf7.png'), dpi=600)
plt.close()

plt.plot(dist1[60:-60],pot1[60:-60],label='Gasolina')
plt.plot(dist2[20:-20],pot2[20:-20],label='Electrico')
plt.title('Grafica 8. potencia del motor vs Distancia recorrida')
pylab.ylabel("Potencia [w]")
pylab.xlabel("Distancia recorrida [m]")
plt.grid()
pylab.legend(loc='lower left') 
pylab.savefig(os.path.join('graf8.png'), dpi=600)
plt.close()


# Análisis del kart

##Comparación del desempeño
Con el fin de encontrar posibles mejoras al desempeño del kart eléctrico buscando que su comportamiento alcance o supere el del vehículo de motor de combustión se desea observar para un mismo recorrido los distintos parámetros de desempeño de cada uno. Para realizar esto se seleccionaron datos de pruebas de cada uno de los vehículos partiendo de una velocidad y posición cercana en un tramo igual de la pista de carreras de Tocancipá.
No se utilizaron los datos de toda la vuelta puesto que el circuito recorrido por cada vehículo vario significativamente en el tramo final. A pesar de esto los tramos tomados son iguales y por tanto permiten analizar el comportamiento del vehículo. El tramo del circuito seleccionado para el análisis se presenta en la gráfica 1.

<img src="graf1.png">

De este tramo en el circuito de pruebas es posible determinar los siguientes datos

<img src="graf2.png">

<img src="graf3.png">

<img src="graf4.png">

En la gráfica 2 es posible apreciar que el vehículo a gasolina toma 48 segundos en recorrer el circuito analizado mientras que el vehículo eléctrico realiza el recorrido en 50.5 segundos. Esta diferencia entre los tiempos recorridos por cada vehículo no se debe a la velocidad inicial puesto que como se puede ver en la gráfica 3 los dos vehículos parten a una velocidad prácticamente igual.

La grafica 2 nos permite apreciar que la curva de velocidad instantánea del vehículo eléctrico tiene en general valores más bajos que la curva del vehículo a gasolina lo que podría ser una de las causas de la diferencia de tiempos entre uno y otro. Esta diferencia de velocidades se ve reflejada en la gráfica 3 donde la aceleración instantánea del vehículo eléctrico es menor a la del vehículo a gasolina.

Algunos factores externos al vehículo también pueden estar influyendo en los tiempos de la prueba puesto que como se observa en el primer pico de velocidad de la gráfica 2 el piloto del vehículo eléctrico empezó a frenar a una distancia menor que el del vehículo a gasolina lo que implicó una distancia de frenado más larga que pudo afectar el desempeño.

Para analizar aquellos factores propios del vehículo se utilizaron los modelos físicos antes desarrollados para determinar el comportamiento del motor de cada vehículo.

##Desempeño de los motores

A continuación, se presentan las gráficas de desempeño de los motores obtenidas para los dos vehículos.

<img src="graf5.png">

<img src="graf6.png">

<img src="graf7.png">

<img src="graf8.png">

La grafica 5 presentan la diferencia de torque generado por el motor para los dos vehículos en donde es posible apreciar que el torque máximo generado por el motor eléctrico es mayor al generado por el motor de combustión interna. lo anterior puede ser explicado por la diferencia de velocidades de trabajo de cada motor como puede ser observado en las gráficas 6 y 7.
El motor de combustión interna alcanza velocidades angulares del orden de 10.000 RPM mientras que el motor eléctrico opera alrededor de 3000 RPM.

Un punto importante que notar es que la velocidad máxima de rotación del motor eléctrico es de 5000 RPM lo que significa que el motor no está trabajando a máxima potencia, unicamente se acerca a 4200 rpm en un tramo recto del circuito. Las causas de esto pueden ser que el vehículo eléctrico no logra transmitir suficiente torque a las ruedas para vencer las fuerzas de arrastre y rodadura del vehículo a velocidades más altas, o que la aceleración es baja y por ende el conductor debe frenar antes de poder alcanzar una velocidad superior.
La segunda causa parece menos probable puesto que como se observa en la gráfica 4 las desaceleraciones del vehículo a gasolina son más altas que las del vehiculó con el motor eléctrico. Teniendo en cuenta que los dos cuentan con el mismo sistema de frenos estos deberían permitir frenar el vehículo eléctrico así este alcanzara velocidades cercanas a las del vehículo a gasolina.

##Opciones de mejora

Cualquiera de las dos causas apreciadas para que el motor eléctrico no se encuentre trabajando a máxima potencia durante el recorrido tienen la misma solución, aumentar el torque transmitido a las ruedas.
Un mayor torque transmitido a las ruedas permitirá alcanzar velocidades mayores pues por el equilibrio de fuerzas sobre el vehículo las fuerzas de arrastre y de rodadura requerían velocidades mayores para igualar a la fuerza transmitida a las ruedas. También un mayor torque implica una aceleración mayor lo que permitiría al vehículo llegar a velocidades más altas antes de tener que frenar. 

Una posible mejora que podría implementarse con el fin de transmitir mayor potencia a las ruedas es aumentar la relación de transmisión del vehículo para que así el motor eléctrico aumente su velocidad de rotación acercándose a su punto de operación de máxima potencia logrando aumentar las velocidades durante el recorrido.

Tambien se puede apreciar en la grafica 8 que la potencia pico del motor electrico es menor a la desarrollada por el motor de combustion interna, esto puede ser causado por la falta de una relacion adecuada para la pista. sin embargo si la correccion de la relacion de transmision no es suficiente se implica que el motor electrico debe ser reemplazado por uno de mayor potencia si se desea que su comportamiento se aproxime al del motor de combustion.


Christian Bohorquez-201424942
